請就所提供的資料進行以下分析：

1.分析持卡戶的特徵(性別、年齡、教育程度...等)對持卡戶是否流失(Attrition_Flag)之影響。

2.請以前述具顯著影響之特徵建立持卡戶是否流失(Attrition_Flag)之機器學習模型，並比較至少三種機器學習模型，分析後請以正確率(Accuracy)、精確率(Precision)、召回率(Recall)比較說明分析結果。

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from scipy import stats
from sklearn import metrics

In [32]:
data=pd.read_csv('BankChurners.csv')
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data=pd.DataFrame(data)
data['Gender'] = labelencoder.fit_transform(data['Gender'])
data['Education_Level'] = labelencoder.fit_transform(data['Education_Level'])
data['Marital_Status'] = labelencoder.fit_transform(data['Marital_Status'])
data['Income_Category'] = labelencoder.fit_transform(data['Income_Category'])
data['Card_Category'] = labelencoder.fit_transform(data['Card_Category'])
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct
0,768805383,1,45,1,3,3,1,2,0,39,5,1,12691.000,777,1144,42
1,818770008,1,49,0,5,2,2,4,0,44,6,1,8256.000,864,1291,33
2,713982108,1,51,1,3,2,1,3,0,36,4,1,3418.000,0,1887,20
3,769911858,1,40,0,4,3,3,4,0,34,3,4,3313.000,2517,1171,20
4,709106358,1,40,1,3,5,1,2,0,21,5,1,4716.000,0,816,28


#1.分析持卡戶的特徵(性別、年齡、教育程度...等)對持卡戶是否流失(Attrition_Flag)之影響。

In [33]:
#1類對多類的卡方檢定

qv3=['Gender','Education_Level','Marital_Status','Income_Category','Card_Category']
alist = []
pvlist = []
name = []
for i in qv3:
  cr=stats.chi2_contingency(pd.crosstab(data['Attrition_Flag'],data[i]))
  pvlist.append(cr[1:2])
  alist.append(cr[0:1])
  name.append(i)
pd.set_option('display.float_format',lambda x:'%.3f'%x)
result=pd.DataFrame(alist,name,columns=['Chi-Square'])
pvlist2 = [item for t in pvlist for item in t] #Tuple to List
result.insert(1,'p_value',pvlist2)
result=result.sort_values(by=['p_value'])[::-1]
result

,Chi-Square,p_value
Card_Category,2.234,0.525
Marital_Status,6.056,0.109
Education_Level,12.511,0.051
Income_Category,12.832,0.025
Gender,13.866,0.000


In [34]:
#1類對多量的t檢定

data_set2=['Customer_Age','Dependent_count','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Credit_Limit','Total_Revolving_Bal','Total_Trans_Amt','Total_Trans_Ct']
name=[]
alist=[]
pvlist=[]
for i in range(len(data_set2)):
  (a,pv)=stats.ttest_ind(data[data_set2[i]][data['Attrition_Flag']==0],data[data_set2[i]][data['Attrition_Flag']==1])
  name.append(data_set2[i])
  alist.append(a)
  pvlist.append(pv)
pd.set_option('display.float_format',lambda x:'%.3f'%x)
result=pd.DataFrame(alist,name,columns=['t_value'])
result.insert(1,'p_value',pvlist)
result=result.sort_values(by=['p_value'])[::-1]
result

,t_value,p_value
Months_on_book,1.377,0.168
Customer_Age,1.832,0.067
Dependent_count,1.911,0.056
Credit_Limit,-2.403,0.016
Total_Relationship_Count,-15.267,0.000
Months_Inactive_12_mon,15.521,0.000
Total_Trans_Amt,-17.211,0.000
Total_Revolving_Bal,-27.435,0.000
Total_Trans_Ct,-40.251,0.000


#2.請以前述具顯著影響之特徵建立持卡戶是否流失(Attrition_Flag)之機器學習模型，並比較至少三種機器學習模型，分析後請以正確率(Accuracy)、精確率(Precision)、召回率(Recall)比較說明分析結果。

#全部有顯著相關的變數
所得、性別、信用卡額度、持有卡片數、未使卡片消費月數、總交易金額、信用卡可週轉額度、總交易筆數

決策樹

In [35]:
X=data[['Gender','Income_Category','Total_Relationship_Count','Months_Inactive_12_mon','Credit_Limit','Total_Trans_Amt','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [36]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=5)
DT= DT.fit(X, y)

In [37]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[ 955  274]
 [ 204 6162]]

              precision    recall  f1-score   support

           0       0.82      0.78      0.80      1229
           1       0.96      0.97      0.96      6366

    accuracy                           0.94      7595
   macro avg       0.89      0.87      0.88      7595
weighted avg       0.94      0.94      0.94      7595



In [38]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[ 314   84]
 [  56 2078]]

              precision    recall  f1-score   support

           0       0.85      0.79      0.82       398
           1       0.96      0.97      0.97      2134

    accuracy                           0.94      2532
   macro avg       0.90      0.88      0.89      2532
weighted avg       0.94      0.94      0.94      2532



隨機森林

In [39]:
X=data[['Gender','Income_Category','Total_Relationship_Count','Months_Inactive_12_mon','Credit_Limit','Total_Trans_Amt','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [40]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=100,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

RandomForestClassifier(max_features='sqrt', oob_score=True, random_state=3)

In [41]:
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[1229    0]
 [   0 6366]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1229
           1       1.00      1.00      1.00      6366

    accuracy                           1.00      7595
   macro avg       1.00      1.00      1.00      7595
weighted avg       1.00      1.00      1.00      7595



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [42]:
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[ 314   84]
 [  37 2097]]

              precision    recall  f1-score   support

           0       0.89      0.79      0.84       398
           1       0.96      0.98      0.97      2134

    accuracy                           0.95      2532
   macro avg       0.93      0.89      0.91      2532
weighted avg       0.95      0.95      0.95      2532



羅吉斯回歸

In [43]:
X=data[['Gender','Income_Category','Total_Relationship_Count','Months_Inactive_12_mon','Credit_Limit','Total_Trans_Amt','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [44]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [45]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[ 413  816]
 [ 180 6186]]

              precision    recall  f1-score   support

           0       0.70      0.34      0.45      1229
           1       0.88      0.97      0.93      6366

    accuracy                           0.87      7595
   macro avg       0.79      0.65      0.69      7595
weighted avg       0.85      0.87      0.85      7595



In [46]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[ 135  263]
 [  67 2067]]

              precision    recall  f1-score   support

           0       0.67      0.34      0.45       398
           1       0.89      0.97      0.93      2134

    accuracy                           0.87      2532
   macro avg       0.78      0.65      0.69      2532
weighted avg       0.85      0.87      0.85      2532



#類別和屬量變數相對更有顯著相關的變數前兩名
所得、性別、信用卡可週轉額度、總交易筆數

決策樹

In [59]:
X=data[['Gender','Income_Category','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [60]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=5)
DT= DT.fit(X, y)

In [61]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[ 693  536]
 [ 243 6123]]

              precision    recall  f1-score   support

           0       0.74      0.56      0.64      1229
           1       0.92      0.96      0.94      6366

    accuracy                           0.90      7595
   macro avg       0.83      0.76      0.79      7595
weighted avg       0.89      0.90      0.89      7595



In [62]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[ 233  165]
 [  79 2055]]

              precision    recall  f1-score   support

           0       0.75      0.59      0.66       398
           1       0.93      0.96      0.94      2134

    accuracy                           0.90      2532
   macro avg       0.84      0.77      0.80      2532
weighted avg       0.90      0.90      0.90      2532



隨機森林

In [63]:
X=data[['Gender','Income_Category','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [64]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=100,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

RandomForestClassifier(max_features='sqrt', oob_score=True, random_state=3)

In [65]:
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[1063  166]
 [ 125 6241]]

              precision    recall  f1-score   support

           0       0.89      0.86      0.88      1229
           1       0.97      0.98      0.98      6366

    accuracy                           0.96      7595
   macro avg       0.93      0.92      0.93      7595
weighted avg       0.96      0.96      0.96      7595



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [66]:
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[ 224  174]
 [ 125 2009]]

              precision    recall  f1-score   support

           0       0.64      0.56      0.60       398
           1       0.92      0.94      0.93      2134

    accuracy                           0.88      2532
   macro avg       0.78      0.75      0.77      2532
weighted avg       0.88      0.88      0.88      2532



羅吉斯回歸

In [67]:
X=data[['Gender','Income_Category','Total_Trans_Ct','Total_Revolving_Bal']]
y=data['Attrition_Flag']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

In [68]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [69]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[ 450  779]
 [ 199 6167]]

              precision    recall  f1-score   support

           0       0.69      0.37      0.48      1229
           1       0.89      0.97      0.93      6366

    accuracy                           0.87      7595
   macro avg       0.79      0.67      0.70      7595
weighted avg       0.86      0.87      0.85      7595



In [70]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[ 155  243]
 [  80 2054]]

              precision    recall  f1-score   support

           0       0.66      0.39      0.49       398
           1       0.89      0.96      0.93      2134

    accuracy                           0.87      2532
   macro avg       0.78      0.68      0.71      2532
weighted avg       0.86      0.87      0.86      2532

